In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from tsfresh.feature_extraction import feature_calculators

def zero_empty(arg):
    if type(arg) == np.ndarray and arg.size > 0:
        return arg
    elif arg.size > 0:
        return arg
    else:
        return np.array([0, 0])
    
def classic_sta_lta(x, length_sta, length_lta):
    
    sta = np.cumsum(x ** 2)

    # Convert to float
    sta = np.require(sta, dtype=np.float)

    # Copy for LTA
    lta = sta.copy()

    # Compute the STA and the LTA
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta

    # Pad zeros
    sta[:length_lta - 1] = 0

    # Avoid division by zero by setting zero values to tiny float
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny

    return sta / lta

def gp_features(iterable, index):
    features = pd.DataFrame(index=index, dtype=np.float32)
    
    peak_heights = [10, 20, 50, 100]

    for segment, sample in enumerate(iterable):
        
        peaks = signal.find_peaks(sample,
                                  height=(100, 1000000),
                                  distance=100)
        
        
        
        features.loc[segment, "num_peaks"] = len(peaks[0])
        features.loc[segment, "max_peak_h"] = max(zero_empty(peaks[1]["peak_heights"]))
        features.loc[segment, "min_peak_h"] = min(zero_empty(peaks[1]["peak_heights"]))
        features.loc[segment, "mean_peak_h"] = np.mean(zero_empty(peaks[1]["peak_heights"]))
        features.loc[segment, "std_peak_h"] = np.std(zero_empty(peaks[1]["peak_heights"]))
        
        features.loc[segment, "max_peak_p"] = max(zero_empty(peaks[0]))
        features.loc[segment, "min_peak_p"] = min(zero_empty(peaks[0]))
        features.loc[segment, "mean_peak_p"] = np.mean(zero_empty(peaks[0]))
        features.loc[segment, "std_peak_p"] = np.std(zero_empty(peaks[0]))
        features.loc[segment, "peak_max_diff"] = max(zero_empty(np.diff(zero_empty(peaks[0]))))
        features.loc[segment, "peak_min_diff"] = min(zero_empty(np.diff(zero_empty(peaks[0]))))
        features.loc[segment, "peak_avg_diff"] = np.mean(zero_empty(np.diff(zero_empty(peaks[0]))))
        features.loc[segment, "peak_std_diff"] = np.std(zero_empty(np.diff(zero_empty(peaks[0]))))
        
        x = pd.Series(sample)
        
        features.loc[segment, 'mean'] = x.mean()
        features.loc[segment, 'std'] = x.std()
        features.loc[segment, 'max'] = x.max()
        features.loc[segment, 'min'] = x.min()

        features.loc[segment, 'mean_change_abs'] = np.mean(np.diff(x))
        features.loc[segment, 'abs_max'] = np.abs(x).max()
        features.loc[segment, 'abs_min'] = np.abs(x).min()

        features.loc[segment, 'std_first_50000'] = x[:50000].std()
        features.loc[segment, 'std_last_50000'] = x[-50000:].std()
        features.loc[segment, 'std_first_10000'] = x[:10000].std()
        features.loc[segment, 'std_last_10000'] = x[-10000:].std()

        features.loc[segment, 'avg_first_50000'] = x[:50000].mean()
        features.loc[segment, 'avg_last_50000'] = x[-50000:].mean()
        features.loc[segment, 'avg_first_10000'] = x[:10000].mean()
        features.loc[segment, 'avg_last_10000'] = x[-10000:].mean()

        features.loc[segment, 'min_first_50000'] = x[:50000].min()
        features.loc[segment, 'min_last_50000'] = x[-50000:].min()
        features.loc[segment, 'min_first_10000'] = x[:10000].min()
        features.loc[segment, 'min_last_10000'] = x[-10000:].min()

        features.loc[segment, 'max_first_50000'] = x[:50000].max()
        features.loc[segment, 'max_last_50000'] = x[-50000:].max()
        features.loc[segment, 'max_first_10000'] = x[:10000].max()
        features.loc[segment, 'max_last_10000'] = x[-10000:].max()
        
        features.loc[segment, 'q95'] = np.quantile(x, 0.95)
        features.loc[segment, 'q99'] = np.quantile(x, 0.99)
        features.loc[segment, 'q05'] = np.quantile(x, 0.05)
        features.loc[segment, 'q01'] = np.quantile(x, 0.01)

        features.loc[segment, 'abs_q95'] = np.quantile(np.abs(x), 0.95)
        features.loc[segment, 'abs_q99'] = np.quantile(np.abs(x), 0.99)
        features.loc[segment, 'abs_q05'] = np.quantile(np.abs(x), 0.05)
        features.loc[segment, 'abs_q01'] = np.quantile(np.abs(x), 0.01)
        
        features.loc[segment, 'mad'] = x.mad()
        features.loc[segment, 'kurt'] = x.kurtosis()
        features.loc[segment, 'skew'] = x.skew()
        features.loc[segment, 'med'] = x.median()
        
        
        features.loc[segment, 'Hilbert_mean'] = np.abs(hilbert(x)).mean()
        features.loc[segment, 'Hann_window_mean'] = (convolve(x, hann(150), mode='same') / sum(hann(150))).mean()
        features.loc[segment, 'classic_sta_lta1_mean'] = classic_sta_lta(x, 500, 10000).mean()
        features.loc[segment, 'classic_sta_lta2_mean'] = classic_sta_lta(x, 5000, 100000).mean()
        features.loc[segment, 'classic_sta_lta3_mean'] = classic_sta_lta(x, 3333, 6666).mean()
        features.loc[segment, 'classic_sta_lta4_mean'] = classic_sta_lta(x, 10000, 25000).mean()
        features.loc[segment, 'classic_sta_lta5_mean'] = classic_sta_lta(x, 50, 1000).mean()
        features.loc[segment, 'classic_sta_lta6_mean'] = classic_sta_lta(x, 100, 5000).mean()
        features.loc[segment, 'classic_sta_lta7_mean'] = classic_sta_lta(x, 333, 666).mean()
        features.loc[segment, 'classic_sta_lta8_mean'] = classic_sta_lta(x, 4000, 10000).mean()
        features.loc[segment, 'Moving_average_700_mean'] = x.rolling(window=700).mean().mean(skipna=True)
        ewma = pd.Series.ewm
        features.loc[segment, 'exp_Moving_average_300_mean'] = (ewma(x, span=300).mean()).mean(skipna=True)
        features.loc[segment, 'exp_Moving_average_3000_mean'] = ewma(x, span=3000).mean().mean(skipna=True)
        features.loc[segment, 'exp_Moving_average_30000_mean'] = ewma(x, span=30000).mean().mean(skipna=True)
        no_of_std = 3
        features.loc[segment, 'MA_700MA_std_mean'] = x.rolling(window=700).std().mean()
        features.loc[segment,'MA_700MA_BB_high_mean'] = (features.loc[segment, 'Moving_average_700_mean'] + no_of_std * features.loc[segment, 'MA_700MA_std_mean']).mean()
        features.loc[segment,'MA_700MA_BB_low_mean'] = (features.loc[segment, 'Moving_average_700_mean'] - no_of_std * features.loc[segment, 'MA_700MA_std_mean']).mean()
        features.loc[segment, 'MA_400MA_std_mean'] = x.rolling(window=400).std().mean()
        features.loc[segment,'MA_400MA_BB_high_mean'] = (features.loc[segment, 'Moving_average_700_mean'] + no_of_std * features.loc[segment, 'MA_400MA_std_mean']).mean()
        features.loc[segment,'MA_400MA_BB_low_mean'] = (features.loc[segment, 'Moving_average_700_mean'] - no_of_std * features.loc[segment, 'MA_400MA_std_mean']).mean()
        features.loc[segment, 'MA_1000MA_std_mean'] = x.rolling(window=1000).std().mean()
        features.drop('Moving_average_700_mean', axis=1, inplace=True)

        features.loc[segment, 'iqr'] = np.subtract(*np.percentile(x, [75, 25]))
        features.loc[segment, 'q999'] = np.quantile(x,0.999)
        features.loc[segment, 'q001'] = np.quantile(x,0.001)
        features.loc[segment, 'ave10'] = stats.trim_mean(x, 0.1)
        
        for windows in [10, 100, 1000, 10000]:
            rolls = [
                ("x_roll_std", x.rolling(windows).std().dropna().values),
                ("x_roll_mean", x.rolling(windows).mean().dropna().values),
                ("x_roll_sum", x.rolling(windows).sum().dropna().values),
                ("x_roll_std_std", x.rolling(windows).std().rolling(windows).std().dropna().values),
                ("x_roll_mean_std", x.rolling(windows).mean().rolling(windows).std().dropna().values),
                ("x_roll_std_mean", x.rolling(windows).std().rolling(windows).mean().dropna().values),
                ("x_roll_mean_mean", x.rolling(windows).mean().rolling(windows).mean().dropna().values),
                ("x_roll_quantile50", x.rolling(windows).quantile(0.5).dropna().values),
                ("x_roll_quantile05", x.rolling(windows).quantile(0.05).dropna().values),
                ("x_roll_quantile90", x.rolling(windows).quantile(0.90).dropna().values)
            ]
            
            for name, roll in rolls:
            
                features.loc[segment, 'ave_%s_%s' % (name, str(windows))] = roll.mean()
                features.loc[segment, 'std_%s_%s' % (name, str(windows))] = roll.std()
                features.loc[segment, 'max_%s_%s' % (name, str(windows))] = roll.max()
                features.loc[segment, 'min_%s_%s' % (name, str(windows))] = roll.min()
                features.loc[segment, 'q01_%s_%s' % (name, str(windows))] = np.quantile(roll, 0.01)
                features.loc[segment, 'q05_%s_%s' % (name, str(windows))] = np.quantile(roll, 0.05)
                features.loc[segment, 'q25_%s_%s' % (name, str(windows))] = np.quantile(roll, 0.25)
                features.loc[segment, 'q50_%s_%s' % (name, str(windows))] = np.quantile(roll, 0.50)
                features.loc[segment, 'q75_%s_%s' % (name, str(windows))] = np.quantile(roll, 0.75)
                features.loc[segment, 'q95_%s_%s' % (name, str(windows))] = np.quantile(roll, 0.95)
                features.loc[segment, 'q99_%s_%s' % (name, str(windows))] = np.quantile(roll, 0.99)
                features.loc[segment, 'median_%s_%s' % (name, str(windows))] = np.median(roll)
                features.loc[segment, 'av_change_abs_%s_%s' % (name, str(windows))] = np.mean(np.diff(roll))
                features.loc[segment, 'av_change_rate_%s_%s' % (name, str(windows))] = np.mean(np.nonzero((np.diff(roll) / roll[:-1]))[0])
                features.loc[segment, 'abs_max_%s_%s' % (name, str(windows))] = np.abs(roll).max()
               
        
        
        features.loc[segment, 'autocorr'] = feature_calculators.autocorrelation(x, 100)
        
        cent, var, skew, kurt = feature_calculators.fft_aggregated(x, [
            {
                "aggtype": "centroid"
            },
            {
                "aggtype": "variance"
            },
            {
                "aggtype": "skew"
            },
            {
                "aggtype": "kurtosis"
            },
        ])
        
        features.loc[segment, 'eig_cent'] = cent[1]
        features.loc[segment, 'eig_var'] = var[1]
        features.loc[segment, 'eig_skew'] = skew[1]
        features.loc[segment, 'eig_kurt'] = kurt[1]
        
        x.name = "hello"
        
        pv, rv, intr, sl, stdr = feature_calculators.linear_trend(x, [{"attr": "pvalue"}
                                                  ,{"attr": "rvalue"}
                                                  ,{"attr": "intercept"}
                                                  ,{"attr": "slope"}
                                                  ,{"attr": "stderr"}])
        
        features.loc[segment, 'lin_pv'] = pv[1]
        features.loc[segment, 'lin_var'] = rv[1]
        features.loc[segment, 'lin_intr'] = intr[1]
        features.loc[segment, 'lin_sl'] = sl[1]
        features.loc[segment, 'lin_stdr'] = stdr[1]
        
        features.loc[segment, 'count_above_mean'] = feature_calculators.count_above_mean(x)
        features.loc[segment, 'count_below_mean'] = feature_calculators.count_below_mean(x)
        
        for peak in peak_heights:
            features.loc[segment, ("num_peaks_%s" % peak)] = feature_calculators.number_peaks(x, peak)
            
            
        for peak in peak_heights:
            peaks = signal.find_peaks(sample,
                              height=(-peak * 10, -peak),
                              distance=40)
            
            
            features.loc[segment, "neg_num_peaks_%s" % peak] = len(peaks[0])
            features.loc[segment, "neg_max_peak_h_%s" % peak] = max(zero_empty(peaks[1]["peak_heights"]))
            features.loc[segment, "neg_min_peak_h_%s" % peak] = min(zero_empty(peaks[1]["peak_heights"]))
            features.loc[segment, "neg_mean_peak_h_%s" % peak] = np.mean(zero_empty(peaks[1]["peak_heights"]))
            features.loc[segment, "neg_std_peak_h_%s" % peak] = np.std(zero_empty(peaks[1]["peak_heights"]))

            features.loc[segment, "neg_max_peak_p_%s" % peak] = max(zero_empty(peaks[0]))
            features.loc[segment, "neg_min_peak_p_%s" % peak] = min(zero_empty(peaks[0]))
            features.loc[segment, "neg_mean_peak_p_%s" % peak] = np.mean(zero_empty(peaks[0]))
            features.loc[segment, "neg_std_peak_p_%s" % peak] = np.std(zero_empty(peaks[0]))
            features.loc[segment, "neg_peak_max_diff_%s" % peak] = max(zero_empty(np.diff(zero_empty(peaks[0]))))
            features.loc[segment, "neg_peak_min_diff_%s" % peak] = min(zero_empty(np.diff(zero_empty(peaks[0]))))
            features.loc[segment, "neg_peak_avg_diff_%s" % peak] = np.mean(zero_empty(np.diff(zero_empty(peaks[0]))))
            features.loc[segment, "neg_peak_std_diff_%s" % peak] = np.std(zero_empty(np.diff(zero_empty(peaks[0]))))
        
        print("Progress: %s / %s" % (segment, len(iterable)), end = "             \r")

    return features